In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import wandb

In [ ]:
dtype = torch.float64

data_files = ['data/raw_data_6.npy',]

# d = [time, x, y, z, dx, dy, dz, roll, pitch, yaw, roll_sp, pitch_sp, yaw_sp, thrust_sp]
d = []
for data_file in data_files:
    with open(data_file, 'rb') as f:
        d.append(torch.tensor(np.load(f), dtype=dtype))

d = torch.cat(d, dim=0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(d[:, 1], d[:, 2], d[:, 3])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(d[1:, 7], '.-')
plt.plot(d[:-1, 10], '.-')
plt.legend(['angle', 'setpoint'])
plt.show()

In [ ]:
step_size = 1

# d = [time, x, y, z, dx, dy, dz, roll, pitch, yaw, roll_sp, pitch_sp, yaw_sp, thrust_sp]
rpy = d[:-step_size, 7:10].clone()
u = d[:-step_size, 10:14].clone()
dvel = d[step_size:, 4:7].clone() - d[:-step_size, 4:7].clone()
drpy = d[step_size:, 7:10].clone() - d[:-step_size, 7:10].clone()
dt = d[step_size:, 0].clone() - d[:-step_size, 0].clone()

good_samples = dt < 1.
rpy = rpy[good_samples, :]
u = u[good_samples, :]
dvel = dvel[good_samples, :]
drpy = drpy[good_samples, :]
dt = dt[good_samples]

b, a = signal.butter(4, .4)

data_raw = torch.cat([rpy, u], dim=1)
# data = torch.tensor(signal.filtfilt(b, a, data_raw.T).T.copy(), dtype=dtype)
data = data_raw

labels_raw = torch.cat([dvel, drpy], dim=1)
# labels = torch.tensor(signal.filtfilt(b, a, labels_raw.T).T.copy(), dtype=dtype)
labels = labels_raw

In [ ]:
fig = plt.figure()
plt.plot(labels_raw[:100, i], '.')
plt.plot(labels[:100, 3], '-')
plt.legend(['raw', 'filtered'])
plt.show()

In [ ]:
dt = torch.mean(dt)
print(dt)

In [ ]:
labels_std = torch.std(labels, dim=0)
print(labels_std)

In [ ]:
print(labels.shape)

In [ ]:
torch.save((data, labels), 'data/px4_data.pt')